In [1]:
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [2]:
df_user = pd.read_csv('./data/train_user.csv')

In [3]:
df_user.head()

,Unnamed: 0,fullVisitorId,is_transaction,transactionRevenue_total,bounces_mean,hits_mean,pageviews_mean,visitNumber_total,is_source_googleplex_freq,is_source_googleplex_rate,...,dow_freq,dow_rate,hour_freq,hour_rate,browser_chrome_freq,browser_chrome_rate,is_mobile_freq,is_mobile_rate,system_google_freq,system_google_rate
0,0,4823595352351,0,0.0,1.0,1.0,1.0,1,0,0.0,...,1,1.0,14,14.0,1,1.0,0,0.0,0,0.0
1,1,5103959234087,0,0.0,0.0,10.0,8.0,1,0,0.0,...,6,6.0,22,22.0,1,1.0,1,1.0,1,1.0
2,2,10278554503158,0,0.0,0.0,11.0,8.0,1,0,0.0,...,4,4.0,5,5.0,1,1.0,0,0.0,0,0.0
3,3,20424342248747,0,0.0,0.0,17.0,13.0,1,0,0.0,...,3,3.0,7,7.0,1,1.0,0,0.0,0,0.0
4,4,26722803385797,0,0.0,0.0,3.0,2.0,1,0,0.0,...,0,0.0,10,10.0,0,0.0,0,0.0,0,0.0


In [4]:
df_user.columns

Index(['Unnamed: 0', 'fullVisitorId', 'is_transaction',
       'transactionRevenue_total', 'bounces_mean', 'hits_mean',
       'pageviews_mean', 'visitNumber_total', 'is_source_googleplex_freq',
       'is_source_googleplex_rate', 'is_source_direct_freq',
       'is_source_direct_rate', 'is_source_google_freq',
       'is_source_google_rate', 'is_medium_referral_freq',
       'is_medium_referral_rate', 'is_medium_none_freq', 'is_medium_none_rate',
       'is_medium_organic_freq', 'is_medium_organic_rate', 'is_bayarea_freq',
       'is_bayarea_rate', 'Africa_freq', 'Africa_rate', 'Americas_freq',
       'Americas_rate', 'Asia_freq', 'Asia_rate', 'Europe_freq', 'Europe_rate',
       'Oceania_freq', 'Oceania_rate', 'next_session_1_adjmean',
       'last_session_1_adjmean', 'q1_freq', 'q1_rate', 'q2_freq', 'q2_rate',
       'q3_freq', 'q3_rate', 'q4_freq', 'q4_rate', 'weekday_freq',
       'weekday_rate', 'h_0_6_freq', 'h_0_6_rate', 'h_7_12_freq',
       'h_7_12_rate', 'h_13_17_freq', 'h_1

In [7]:
len(df_user)

723803

# fit model

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

## split

In [32]:
df_user_train = df_user.sample(frac=0.67, random_state=43)
len(df_user_train)

484948

## from X_train, y_train, get balanced sample

In [33]:
df_user_0 = df_user_train[df_user_train['is_transaction']==1]
len(df_user_0)

6911

In [34]:
df_user_1 = df_user_train[df_user_train['is_transaction']==0].sample(
    n=len(df_user_0), random_state=3
)
len(df_user_1)

6911

In [37]:
df_user_balanced = pd.concat([df_user_0,df_user_1])
df_user_balanced.columns

Index(['Unnamed: 0', 'fullVisitorId', 'is_transaction',
       'transactionRevenue_total', 'bounces_mean', 'hits_mean',
       'pageviews_mean', 'visitNumber_total', 'is_source_googleplex_freq',
       'is_source_googleplex_rate', 'is_source_direct_freq',
       'is_source_direct_rate', 'is_source_google_freq',
       'is_source_google_rate', 'is_medium_referral_freq',
       'is_medium_referral_rate', 'is_medium_none_freq', 'is_medium_none_rate',
       'is_medium_organic_freq', 'is_medium_organic_rate', 'is_bayarea_freq',
       'is_bayarea_rate', 'Africa_freq', 'Africa_rate', 'Americas_freq',
       'Americas_rate', 'Asia_freq', 'Asia_rate', 'Europe_freq', 'Europe_rate',
       'Oceania_freq', 'Oceania_rate', 'next_session_1_adjmean',
       'last_session_1_adjmean', 'q1_freq', 'q1_rate', 'q2_freq', 'q2_rate',
       'q3_freq', 'q3_rate', 'q4_freq', 'q4_rate', 'weekday_freq',
       'weekday_rate', 'h_0_6_freq', 'h_0_6_rate', 'h_7_12_freq',
       'h_7_12_rate', 'h_13_17_freq', 'h_1

In [42]:
X_train = df_user_balanced.drop(['is_transaction', 'fullVisitorId', 'transactionRevenue_total'], axis=1)
y_train = df_user_balanced['is_transaction']

In [49]:
df_user_test = df_user.loc[~df_user.index.isin(df_user_train.index), :]

In [50]:
X_test = df_user_test.drop(['is_transaction', 'fullVisitorId', 'transactionRevenue_total'], axis=1)
y_test = df_user_test['is_transaction']

In [56]:
len(X_test)

238855

In [55]:
reg = LogisticRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

precision_recall_fscore_support(y_test, y_pred, average='macro')

# fit whole dataset

In [25]:
X_test_whole = df_user.drop(['is_transaction', 'fullVisitorId', 'transactionRevenue_total'], axis=1)
y_test_whole = df_user['is_transaction']

In [26]:
y_pred_whole = reg.predict(X_test_whole)

In [27]:
precision_recall_fscore_support(y_test_whole, y_pred_whole, average='macro') # ('precision', 'recall', 'f-score')

(0.5897808697000054, 0.950909664226554, 0.6355792740456989, None)

In [28]:
confusion_matrix(y_test_whole, y_pred_whole) 

array([[669304,  44385],
       [   364,   9750]])

## Stacking

In [1]:
from stacked_estimator import *